In [1]:
import esm
import torch
import pandas as pd
import tqdm

/home/digilab/miniconda3/envs/tp/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
model.eval()

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /home/digilab/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to /home/digilab/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt


ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bia

In [3]:
df = pd.read_csv('data/OsmoticStress_with_binary_positions.csv')

In [4]:
def generate_embeddings(model, alphabet, sequences, batch_size=32):
    embeddings = []
    
    batch_converter = alphabet.get_batch_converter()
    sequence_batches = [sequences[i:i + batch_size] for i in range(0, len(sequences), batch_size)]

    for batch in tqdm.tqdm(sequence_batches, desc="Generating Embeddings"):
        data = [(0, seq) for seq in batch]
        batch_labels, batch_strs, batch_tokens = batch_converter(data)

        with torch.no_grad():
            results = model(batch_tokens, repr_layers=[33])
            token_embeddings = results['representations'][33]

        # Average embeddings across all tokens and convert to numpy
        averaged_embeddings = token_embeddings.mean(dim=1).numpy()
        embeddings.extend(averaged_embeddings)
    
    return np.vstack(embeddings)

sequences = df['full_sequence'].tolist()
df['Full_embedding'] = generate_embeddings(model, alphabet, sequences)

Generating Embeddings:   0%|          | 0/32709 [00:00<?, ?it/s]

Generating Embeddings:   0%|          | 44/32709 [06:56<85:55:35,  9.47s/it] 


KeyboardInterrupt: 

In [ ]:
df.to_csv('data/OsmoticStress_with_binary_positions_and_embeddings.csv', index=False)